In [9]:
# Import warnings filter
from warnings import simplefilter
# Ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss
import scipy
from scipy.interpolate import interp1d 

from sklearn.ensemble import (
    RandomForestClassifier, 
    ExtraTreesClassifier, 
    GradientBoostingClassifier, 
    AdaBoostClassifier, 
    VotingClassifier, 
    BaggingClassifier
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import (
    LogisticRegression, 
    RidgeCV, 
    LassoCV, 
    Ridge, 
    Lasso
)
from sklearn.naive_bayes import (
    MultinomialNB, 
    GaussianNB
)
from sklearn.svm import (
    LinearSVC, 
    SVC
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import (
    GridSearchCV, 
    train_test_split, 
    cross_val_score, 
    cross_validate, 
    cross_val_predict, 
    StratifiedKFold, 
    LeaveOneOut, 
    KFold, 
    RepeatedStratifiedKFold
)
from sklearn.feature_selection import (
    SelectFromModel, 
    RFECV
)
from sklearn.discriminant_analysis import (
    LinearDiscriminantAnalysis, 
    QuadraticDiscriminantAnalysis
)
from sklearn.metrics import (
    accuracy_score, 
    confusion_matrix, 
    classification_report, 
    roc_curve, 
    auc, 
    roc_auc_score, 
    fbeta_score, 
    make_scorer, 
    matthews_corrcoef
)
from sklearn.manifold import TSNE
from sklearn.preprocessing import (
    StandardScaler, 
    MinMaxScaler
)
from sklearn.utils import resample
from sklearn import preprocessing, metrics, datasets

from imblearn.over_sampling import (
    SMOTE, 
    ADASYN, 
    RandomOverSampler, 
    SVMSMOTE, 
    BorderlineSMOTE, 
    SMOTENC, 
    KMeansSMOTE
)
from imblearn.under_sampling import (
    ClusterCentroids, 
    RepeatedEditedNearestNeighbours, 
    EditedNearestNeighbours, 
    AllKNN, 
    CondensedNearestNeighbour, 
    TomekLinks, 
    InstanceHardnessThreshold, 
    NearMiss, 
    NeighbourhoodCleaningRule, 
    OneSidedSelection, 
    RandomUnderSampler
)
from imblearn.ensemble import (
    BalancedBaggingClassifier, 
    BalancedRandomForestClassifier, 
    EasyEnsembleClassifier, 
    RUSBoostClassifier
)
from imblearn.metrics import (
    classification_report_imbalanced, 
    sensitivity_score, 
    specificity_score
)

import xgboost as xgb
#import pymrmr
import pickle
import itertools
from collections import Counter

In [ ]:
#x = 152641 (dataset ID); data_x = Normalized bulk expression measurments; labs_x = classification labels (0=non covid and 2=covid); top_genes_clusterX_x = list of gene names which are markers for the current cluster X (0, 1 or 6)
data_152641=pd.read_table('Normalized data_152641.txt')
labs_152641=pd.read_table('Labels_152641.txt')
labs_152641=labs_152641['V1'].astype('category').cat.codes
top_genes_cluster0_152641=pd.read_table('topcluster_0_152641.txt')
top_genes_cluster1_152641=pd.read_table('topcluster_1_152641.txt')
top_genes_cluster6_152641=pd.read_table('topcluster_6_152641.txt')

In [ ]:
data_152075=pd.read_table('Normalized data_152075.txt')
labs_152075=pd.read_table('Labels_152075.txt')
labs_152075=labs_152075['V1'].astype('category').cat.codes
top_genes_cluster0_152075=pd.read_table('topcluster_0_152075.txt')
top_genes_cluster1_152075=pd.read_table('topcluster_1_152075.txt')
top_genes_cluster6_152075=pd.read_table('topcluster_6_152075.txt')

In [ ]:
data_152418=pd.read_table('Normalized data_152418.txt')
labs_152418=pd.read_table('Labels_152418.txt')
labs_152418=labs_152418['V1'].astype('category').cat.codes
top_genes_cluster0_152418=pd.read_table('topcluster_0_152418.txt')
top_genes_cluster1_152418=pd.read_table('topcluster_1_152418.txt')
top_genes_cluster6_152418=pd.read_table('topcluster_6_152418.txt')

In [ ]:
print("Sample sizes: ", "GSE152641 - ", data_152641.shape[0], "; ","GSE152075 - ", data_152075.shape[0], "; ", "GSE152418 - ", data_152418.shape[0], "; " )

Sample sizes:  GSE152641 -  86 ;  GSE152075 -  484 ;  GSE152418 -  34 ; 


In [ ]:
print("Data distribution: ", "GSE152641 - \n", labs_152641.value_counts())
print("Data distribution: ", "GSE152075 - \n", labs_152075.value_counts())
print("Data distribution: ", "GSE152418 - \n", labs_152418.value_counts())

Data distribution:  GSE152641 - 
 0    62
1    24
dtype: int64
Data distribution:  GSE152075 - 
 1    430
0     54
dtype: int64
Data distribution:  GSE152418 - 
 0    17
1    17
dtype: int64


In [ ]:
def model_fit_and_validate(X,y,n,sampling,model,genes,n_splits,name):
    tprs = []
    base_fpr = np.linspace(0, 1, 101)
    idx = np.arange(0, len(y))

    plt.figure(figsize=(5, 5))
    rskf=RepeatedStratifiedKFold(n_splits=n_splits,n_repeats=n)
    i=0
    auc=[];sensitivity=[];specificity=[];m=[];accuracy=[];cols=[]
    for train_index,test_index in rskf.split(X,y):
        i=i+1
        print(i,end=",")
        X_train,X_test=X.iloc[train_index],X.iloc[test_index]
        y_train,y_test=y.iloc[train_index],y.iloc[test_index]
        scaler = MinMaxScaler()
        X_train = pd.DataFrame(scaler.fit_transform(X_train),columns = X_train.columns)
        X_test = pd.DataFrame(scaler.transform(X_test),columns = X_test.columns)
        X_train_sel=X_train[genes]
        X_samp,y_samp=sampling.fit_resample(X_train_sel,y_train.ravel())
        X_samp = pd.DataFrame(X_samp, columns = X_train_sel.columns)
        
        #model fitting
        if model=="RF":
            sen, spe, auc_score,feat,acc,mcc,fpr,tpr,thresholds=RF(X_samp,y_samp,X_test,y_test)
        elif model=="LOGIT":
            sen, spe, auc_score,feat,acc,mcc,fpr,tpr,thresholds=LOGIT(X_samp,y_samp,X_test,y_test)
        
        plt.plot(fpr, tpr, 'b', alpha=0.05)
        tpr = interp(base_fpr, fpr, tpr)
        tpr[0] = 0.0
        tprs.append(tpr)
        auc.append(auc_score)
        sensitivity.append(sen)
        specificity.append(spe)
        accuracy.append(acc)
        m.append(mcc)
        cols.append(feat)
    tprs = np.array(tprs)
    mean_tprs = tprs.mean(axis=0)
    std = tprs.std(axis=0)

    tprs_upper = np.minimum(mean_tprs + std, 1)
    tprs_lower = mean_tprs - std
    
    l_ci=mean_confidence_interval(auc)[1]
    r_ci=mean_confidence_interval(auc)[2]
    plt.plot(base_fpr, mean_tprs, 'b',color='darkorange',lw=2,label=  '(AUC = %0.3f (CI: %0.3f - %0.3f)'  % (round(np.mean(auc),3),round(l_ci,3),round(r_ci,3)))
    plt.legend(loc='lower right')
    plt.title(name)
    plt.fill_between(base_fpr, tprs_lower, tprs_upper, color='grey', alpha=0.3)
    plt.savefig('roc.pdf')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([-0.01, 1.01])
    plt.ylim([-0.01, 1.01])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.axes().set_aspect('equal', 'datalim')
    plt.show()
    return auc, sensitivity, specificity,accuracy,m,cols
        

In [ ]:
#change this portion to run different combinations of samplers and classifiers using the top genes from cluster number x (x=0,1,6). Below, I ran an undersampler named cluster centroids and trained using a random forest classifier using top genes from cluster 6 for the dataset GSE152641
#GSE152641 using cluster 6 features
cc=ClusterCentroids(random_state=42)
name="ROC plot GSE152641 using top features from cluster 6"
auc, sensitivity, specificity,accuracy,m,cols=model_fit_and_validate(data_152641, labs_152641, 4, cc, "RF", top_genes_cluster6_152641['V1'], 5, name)

In [ ]:
print("GSE152641 results using cluster 6 markers")
l=[np.mean(auc),mean_confidence_interval(auc)[1],mean_confidence_interval(auc)[2],np.mean(sensitivity),mean_confidence_interval(sensitivity)[1],mean_confidence_interval(sensitivity)[2],np.mean(specificity),mean_confidence_interval(specificity)[1],mean_confidence_interval(specificity)[2],np.mean(m),mean_confidence_interval(m)[1],mean_confidence_interval(m)[2]]
row=pd.Series(l,['auc','auc_lci','auc_rci','sen','sen_lci','sen_rci','spe','spe_lci','spe_rci','m','m_lci','m_rci'])
df=pd.DataFrame()
df=df.append([row],ignore_index=True)
#df.to_csv('/data/guntaash/Normalized_Data_for_training/GSE152075_results/GSE152075_CC_LOGIT_30.csv',index=False)
flat_list = [item for sublist in cols for item in sublist]
x=Counter(flat_list)
print(df)
#with open('/data/guntaash/Normalized_Data_for_training/GSE152075_results/GSE152075_CC_LOGIT_Counter.txt', 'w') as f:
    #print(x, file=f)

GSE152641 results using cluster 6 markers
        auc   auc_lci   auc_rci    sen   sen_lci   sen_rci       spe  \
0  0.885545  0.841598  0.929492  0.785  0.687152  0.882848  0.792628   

    spe_lci   spe_rci         m     m_lci     m_rci  
0  0.736071  0.849186  0.548564  0.433613  0.663515  


In [ ]:
#GSE152075 using cluster 06features
cc=SMOTE(random_state=42)
name="ROC plot GSE152075 using top features from cluster 6"
auc, sensitivity, specificity,accuracy,m,cols=model_fit_and_validate(data_152075, labs_152075, 3, cc, "RF", top_genes_cluster6_152075['V1'], 10,name)

In [ ]:
print("GSE152075 results using cluster 0 markers")
l=[np.mean(auc),mean_confidence_interval(auc)[1],mean_confidence_interval(auc)[2],np.mean(sensitivity),mean_confidence_interval(sensitivity)[1],mean_confidence_interval(sensitivity)[2],np.mean(specificity),mean_confidence_interval(specificity)[1],mean_confidence_interval(specificity)[2],np.mean(m),mean_confidence_interval(m)[1],mean_confidence_interval(m)[2]]
row=pd.Series(l,['auc','auc_lci','auc_rci','sen','sen_lci','sen_rci','spe','spe_lci','spe_rci','m','m_lci','m_rci'])
df=pd.DataFrame()
df=df.append([row],ignore_index=True)
#df.to_csv('/data/guntaash/Normalized_Data_for_training/GSE152075_results/GSE152075_CC_LOGIT_30.csv',index=False)
flat_list = [item for sublist in cols for item in sublist]
x=Counter(flat_list)
print(df)
#with open('/data/guntaash/Normalized_Data_for_training/GSE152075_results/GSE152075_CC_LOGIT_Counter.txt', 'w') as f:
    #print(x, file=f)

GSE152075 results using cluster 0 markers
        auc   auc_lci   auc_rci       sen   sen_lci   sen_rci       spe  \
0  0.862416  0.833658  0.891174  0.982171  0.973139  0.991202  0.405556   

    spe_lci   spe_rci         m     m_lci     m_rci  
0  0.326829  0.484282  0.503889  0.425084  0.582694  


In [ ]:
#GSE152418 using cluster 1 features
cc=ClusterCentroids(random_state=42)
name="ROC plot GSE152418 using top features from cluster 6"
auc, sensitivity, specificity,accuracy,m,cols=model_fit_and_validate(data_152418, labs_152418, 4, cc, "RF", top_genes_cluster6_152418['V1'], 5, name)

In [ ]:
#GSE152418 using cluster 1 features
cc=ClusterCentroids(random_state=42)
name="ROC plot GSE152418 using top features from cluster 6"
auc, sensitivity, specificity,accuracy,m,cols=model_fit_and_validate(data_152418, labs_152418, 4, cc, "LOGIT", top_genes_cluster6_152418['V1'], 5, name)

In [ ]:
print("GSE152418 results using cluster 6 markers")
l=[np.mean(auc),mean_confidence_interval(auc)[1],mean_confidence_interval(auc)[2],np.mean(sensitivity),mean_confidence_interval(sensitivity)[1],mean_confidence_interval(sensitivity)[2],np.mean(specificity),mean_confidence_interval(specificity)[1],mean_confidence_interval(specificity)[2],np.mean(m),mean_confidence_interval(m)[1],mean_confidence_interval(m)[2]]
row=pd.Series(l,['auc','auc_lci','auc_rci','sen','sen_lci','sen_rci','spe','spe_lci','spe_rci','m','m_lci','m_rci'])
df=pd.DataFrame()
df=df.append([row],ignore_index=True)
#df.to_csv('/data/guntaash/Normalized_Data_for_training/GSE152075_results/GSE152075_CC_LOGIT_30.csv',index=False)
flat_list = [item for sublist in cols for item in sublist]
x=Counter(flat_list)
print(df)
#with open('/data/guntaash/Normalized_Data_for_training/GSE152075_results/GSE152075_CC_LOGIT_Counter.txt', 'w') as f:
    #print(x, file=f)

GSE152418 results using cluster 6 markers
        auc   auc_lci   auc_rci       sen   sen_lci   sen_rci    spe  \
0  0.865278  0.798384  0.932172  0.845833  0.752612  0.939055  0.725   

    spe_lci   spe_rci         m     m_lci     m_rci  
0  0.586907  0.863093  0.588804  0.418019  0.759589  


In [ ]:
def RF(X_train,y_train,X_test,y_test):
    brf=RandomForestClassifier(class_weight="balanced_subsample")
    X_reduced=feature_reduction_using_RFECV(brf,X_train,y_train)
    param_grid = { 
    'n_estimators':[50,100,200,300,400],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [2,3,5,7,9,11],
}
    clf_brf = GridSearchCV(brf, param_grid, cv=3,scoring="roc_auc",n_jobs=110)
    best_model_brf=clf_brf.fit(X_reduced,y_train)
    best_model_brf.fit(X_reduced, y_train)
    y_probs = best_model_brf.predict_proba(X_test[X_reduced.columns])[:,1]
    fp_rate, tp_rate, thresholds = roc_curve(y_test, y_probs,pos_label=1)
    y_test_predictions=best_model_brf.predict(X_test[X_reduced.columns])
    sen= sensitivity_score(y_test, y_test_predictions, pos_label=1, average='binary')
    spe=specificity_score(y_test,y_test_predictions,pos_label=1,average='binary')
    acc=accuracy_score(y_test,y_test_predictions)
    mcc=matthews_corrcoef(y_test, y_test_predictions)
    auc_score=metrics.auc(fp_rate,tp_rate)
    return sen, spe, auc_score, X_reduced.columns,acc,mcc,fp_rate,tp_rate,thresholds

In [ ]:
def LOGIT(X_train,y_train,X_test,y_test):
    logit=LogisticRegression(class_weight="balanced")
    X_reduced=feature_reduction_using_RFECV(logit,X_train,y_train)
    penalty = ['l1', 'l2']
    # Create regularization hyperparameter space
    C = np.logspace(-3, 3, 10)
    # Create hyperparameter options
    hyperparameters = dict(C=C, penalty=penalty)
    clf_logit = GridSearchCV(logit, hyperparameters, cv=3, scoring="roc_auc",n_jobs=110)
    best_model_logit = clf_logit.fit(X_reduced,y_train)
    best_model_logit.fit(X_reduced, y_train)
    y_probs = best_model_logit.predict_proba(X_test[X_reduced.columns])[:,1]
    fp_rate, tp_rate, thresholds = roc_curve(y_test, y_probs)
    auc_score=metrics.auc(fp_rate, tp_rate)
    y_test_predictions=best_model_logit.predict(X_test[X_reduced.columns])
    sen= sensitivity_score(y_test, y_test_predictions, pos_label=1, average='binary')
    spe=specificity_score(y_test,y_test_predictions,pos_label=1,average='binary')
    acc=accuracy_score(y_test,y_test_predictions)
    mcc=matthews_corrcoef(y_test, y_test_predictions)
    return sen, spe, auc_score, X_reduced.columns,acc,mcc,fp_rate,tp_rate,thresholds

In [ ]:

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

In [ ]:
def feature_reduction_using_RFECV(model,X,y):
    feature_list=[]
    np.random.seed(315)
    
    rfecv = RFECV(estimator=model, step=1, cv=3, scoring='roc_auc',n_jobs=110)
    rfecv.fit(X, y)
    new_X=X[X.columns[rfecv.support_==True]]
    return new_X